# Learning TFIDF


In [5]:
import random
import numpy as np
import re
import time
import pickle
import unidecode
from sklearn.feature_extraction.text import TfidfVectorizer

from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models.phrases import Phrases, Phraser

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/maiapolo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Preparing: opening auxiliary files and defining functions

Function that cleans texts:

In [6]:
def clean(resulta):   
    import copy 
   
    result = copy.deepcopy(resulta)
    
    result=result.lower()
    result=" "+result #colocando espaço no começo
    result=re.sub('\d', ' ', result)
    result=result.replace("lei ", "lei_")
    result=result.replace("lei nº ", "lei_")
    result=result.replace("lei n.º" ,"lei_")
    result=result.replace("lei estadual nº ", "lei_") 
    result=result.replace("lei federal nº ", "lei_") 
    result=result.replace("lei municipal nº ", "lei_")
    result=result.replace("fl. ", "fls. ")
    result=result.replace("fls. ", "fls_") 
    result=result.replace("p. ", "pp. ")
    result=result.replace("pp. ", "pp_")
    result=result.replace("art. ", "art_") 
    result=result.replace("artigo ", "art_")
    result=result.replace("inciso ", "inciso_") 
    result=result.replace("nº ", "nº_")
    result=result.replace("n° ", "nº_")
    result=result.replace("º ", "º")
    result=result.replace("ª ", "ª")
    result=result.replace("oab ", "oab_")
    result=result.replace("r$ ", "r$_")
    result=result.replace("\n", " ")
    result=result.replace("dr ", "dr_")
    result=result.replace("dr. ", "dr_")
    result=result.replace("dra ", "dr_")
    result=result.replace("dra. ", "dr_")
    result=result.replace("adv: ", "adv_") 
    
    result=result.replace("/", " ")
    result=result.replace("|", " ")
    result=result.replace("+", " ")
    result=result.replace(".", " ")
    result=result.replace(",", " ")
    result=result.replace(":", " ")
    result=result.replace(";", " ")
    result=result.replace("!", " ")
    result=result.replace("?", " ")
    result=result.replace(">", " ")
    result=result.replace("=", " ")
    result=result.replace("§", " ")
    result=result.replace(" - ", " ")
    result=result.replace(" _ ", " ")
    result=result.replace("&", " ")
    result=result.replace("*", " ")
    result=result.replace("(", " ")
    result=result.replace(")", " ")
    result=result.replace("ª", " ")
    result=result.replace("º", " ")
    result=result.replace("%", " ")
    result=result.replace("[", " ")
    result=result.replace("]", " ")
    result=result.replace("{", " ")
    result=result.replace("}", " ")
    result=result.replace("'", " ")
    result=result.replace('"', " ")
    result=result.replace("“", " ")
    result=result.replace("”", " ")
    result=re.sub(' +', ' ', result)
    result=result+" " #colocando espaço no fim
    
    return(result)

Opening unlabeled motions dataset:

In [7]:
with open("bases/mov_treino.txt", "rb") as fp:   # Unpickling
    mov = pickle.load(fp)

Cleaning texts and storing them in a list:

In [8]:
start_time = time.time()
texts_mov=[]

for i in range(len(mov)):
    for j in range(len(mov[i])):
        texts_mov.append(clean(unidecode.unidecode(mov[i][j][1])))
print(round((time.time() - start_time)/60,2),"minutos")

3.38 minutos


In [9]:
len(texts_mov)

2904255

Opening algorithms trained in the  Word2/Vec & Doc2Vec notebook:

In [10]:
bigrams=Phrases.load('modelos/bigrams_mov')
bibigrams=Phrases.load('modelos/bibigrams_mov')

Tokenizer

In [11]:
#Tokenizador de textos
stop_words = set(stopwords.words('portuguese'))

def tokenize_tfidf(txt):
    texto=txt
    texto=texto.split(' ') 
    texto=bibigrams[bigrams[texto]]
    tokens=[]
    for t in texto:
        if t not in stop_words: tokens.append(t)
        else: pass
 
    return(tokens)

TFIDF

In [12]:
start_time = time.time()

vectorizer = TfidfVectorizer(min_df=1, max_features=2000,
                             ngram_range=(1, 1), tokenizer=tokenize_tfidf)
vectorizer.fit(texts_mov)

(time.time() - start_time)/60

/usr/local/lib/python3.5/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


10.167833630243937

In [13]:
filename = 'modelos/tfidf_mov_V1.sav'
pickle.dump(vectorizer, open(filename, 'wb'))